In [1]:
import pandas as pd

In [2]:
# import data tables
df_pm = pd.read_csv('/Users/clarissaclark/SIADS_Milestone_1/df_2010_2024.csv')
df_vehicles = pd.read_csv('/Users/clarissaclark/SIADS_Milestone_1/vehicle_df')

In [3]:
df_vehicles

,Data Year,County,Fuel Category,Number of Vehicles
0,2010,Alameda,EV,20
1,2010,Alameda,Fossil Fuel,885402
2,2010,Alpine,EV,0
3,2010,Alpine,Fossil Fuel,1041
4,2010,Amador,EV,1
...,...,...,...,...
1647,2023,Ventura,Fossil Fuel,651851
1648,2023,Yolo,EV,6757
1649,2023,Yolo,Fossil Fuel,152560
1650,2023,Yuba,EV,961


In [4]:
# seperating creating ev/gas columns so its easier to compare with pm2.5 values 
df_ev_vehicles = df_vehicles[df_vehicles['Fuel Category'] == 'EV']
df_ev_vehicles = df_ev_vehicles.rename(columns={'Number of Vehicles': 'Number of EV'})
df_ev_vehicles = df_ev_vehicles.drop(columns=['Fuel Category'])
df_ev_vehicles

,Data Year,County,Number of EV
0,2010,Alameda,20
2,2010,Alpine,0
4,2010,Amador,1
6,2010,Butte,0
8,2010,Calaveras,0
...,...,...,...
1642,2023,Tulare,3837
1644,2023,Tuolumne,698
1646,2023,Ventura,31980
1648,2023,Yolo,6757


In [5]:
df_gas_vehicles = df_vehicles[df_vehicles['Fuel Category'] == 'Fossil Fuel']
df_gas_vehicles = df_gas_vehicles.rename(columns={'Number of Vehicles': 'Number of Gas Cars'})
df_gas_vehicles = df_gas_vehicles.drop(columns=['Fuel Category'])
df_gas_vehicles

,Data Year,County,Number of Gas Cars
1,2010,Alameda,885402
3,2010,Alpine,1041
5,2010,Amador,30480
7,2010,Butte,133169
9,2010,Calaveras,39636
...,...,...,...
1643,2023,Tulare,331744
1645,2023,Tuolumne,54369
1647,2023,Ventura,651851
1649,2023,Yolo,152560


In [6]:
# recombining df_vehicles 
df_vehicles = pd.merge(df_gas_vehicles, df_ev_vehicles, on=['Data Year', 'County'], how='inner')
df_vehicles.shape

(826, 4)

In [7]:
df_pm.shape

(766, 4)

In [8]:
# df_vehicles.shape = df_pm.shape + 60, so ...

counties = pd.DataFrame({
    'County_df_vehicles': df_vehicles['County'],
    'County_df_pm': df_pm['County']
})
counties.head()
# Get unique counties from both DataFrames
counties_vehicles = set(df_vehicles['County'])
counties_pm = set(df_pm['County'])

# Find differences
only_in_vehicles = counties_vehicles - counties_pm
only_in_pm = counties_pm - counties_vehicles

print("Only in df_vehicles:", only_in_vehicles)
print("Only in df_pm:", only_in_pm)

# do we want to drop these values ? i will merge on left for now, idk if we will need these since they don't have pm2.5 data at all. we could estimate these values by comparing the counties around them that do report pm2.5 values. 

Only in df_vehicles: {'Out Of State', 'Tuolumne', 'Lassen', 'Amador', 'Modoc', 'Sierra', 'Yuba'}
Only in df_pm: set()


In [9]:
# merging pm and vehicle datatables
df_pm_vehicles = pd.merge(df_pm, df_vehicles, on=['Data Year', 'County'], how='left')
df_pm_vehicles

,Data Year,County,Average PM 2.5/County,Number of Site IDs/County,Number of Gas Cars,Number of EV
0,2010,Alameda,8.165623,5.0,885402.0,20.0
1,2010,Butte,8.776451,3.0,133169.0,0.0
2,2010,Calaveras,7.380349,1.0,39636.0,0.0
3,2010,Colusa,6.054913,1.0,14610.0,0.0
4,2010,Contra Costa,7.606920,1.0,666294.0,10.0
...,...,...,...,...,...,...
761,2024,Tehama,3.965116,1.0,NaN,NaN
762,2024,Trinity,5.718966,1.0,NaN,NaN
763,2024,Tulare,9.256527,3.0,NaN,NaN
764,2024,Ventura,6.063194,5.0,NaN,NaN


In [10]:
df_pm_vehicles.to_csv('/Users/clarissaclark/SIADS_Milestone_1/PM2.5_Vehicles.csv', index=False)

In [11]:
# now let's combine that dataframe with population 
df_pop = pd.read_csv('/Users/clarissaclark/SIADS_Milestone_1/California_Population.csv') 
# clean df_pop county column to match df_pm_vehicle county column 
df_pop['County'] = df_pop['County'].str.strip()
df_pop['County'] = df_pop['County'].str.title()
df_pop['County'] = df_pop['County'].str.replace(r'\s+', ' ', regex=True) 

In [12]:
# Get unique counties from both DataFrames
counties_pop = set(df_pop['County'])
counties_pm_vehicles = set(df_pm_vehicles['County'])

# Find differences
only_in_pop = counties_pop - counties_pm_vehicles
only_in_pm_vehicles = counties_pm_vehicles - counties_pop

print("Only in df_pop:", only_in_pop)
print("Only in df_pm_vehicles:", only_in_pm_vehicles)
# for now, we will merge on left for df_pm_vehicles 

Only in df_pop: {'Tuolumne', 'Lassen', 'Amador', 'Modoc', 'Sierra', 'Yuba', 'State Total'}
Only in df_pm_vehicles: {'Alpine'}


In [13]:
df_pm_vehicles_pop= pd.merge(df_pm_vehicles, df_pop, on=['County', 'Data Year'], how='left')
df_pm_vehicles_pop.rename(columns={'Z Score':'Population Z Score'}, inplace=True)
df_pm_vehicles_pop

,Data Year,County,Average PM 2.5/County,Number of Site IDs/County,Number of Gas Cars,Number of EV,Population,Population Z Score
0,2010,Alameda,8.165623,5.0,885402.0,20.0,1510271.0,0.033519
1,2010,Butte,8.776451,3.0,133169.0,0.0,220000.0,-0.216348
2,2010,Calaveras,7.380349,1.0,39636.0,0.0,45578.0,-0.250126
3,2010,Colusa,6.054913,1.0,14610.0,0.0,21419.0,-0.254805
4,2010,Contra Costa,7.606920,1.0,666294.0,10.0,1049025.0,-0.055803
...,...,...,...,...,...,...,...,...
761,2024,Tehama,3.965116,1.0,NaN,NaN,64308.0,-0.246499
762,2024,Trinity,5.718966,1.0,NaN,NaN,15915.0,-0.255871
763,2024,Tulare,9.256527,3.0,NaN,NaN,478918.0,-0.166208
764,2024,Ventura,6.063194,5.0,NaN,NaN,823863.0,-0.099407


In [14]:
df_pm_vehicles_pop.to_csv('/Users/clarissaclark/SIADS_Milestone_1/PM2.5_Vehicles_Population.csv', index=False)